# Portfolio equation

## Computational problem

<br>
<br>

Considering an interval of time \[$t_{i}$, ..., $t_{f}$ \]

<br>


Given N assets describing by its mean returns, with $\mu_{it}$ being the return of the i-th asset at time t.

<br>
<br>

One usually deal with $<\omega_{nt}>$, the mean invertions in n-th asset at time t.

<br>

However, it is easier to work with proportions under the following normalization:

<br>

 $\sum_{n}<\omega_{nt}> = K$  for all t, and K being the total investment.

<br>


Thus, $\omega_{nt}=\frac{<\omega_{nt}>}{K}$ is the mean proportion of invertion in the n-th asset at time t. 

<br>
<br>

<br>
<br>

**The problem is ...**

Find an array of vectors **{{$\omega_{1t_1}, ... \omega_{Nt_1}$}, ... , {$\omega_{1t_f}, ... \omega_{Nt_f}$}}** so that the following Hamiltonian gets a minimum value:

<br>
<br>

(1) $H=\sum_{t}-\mu _{t}^{T}\omega_{t}+0.5\gamma \omega_{t}^{T}\sum_{t}\omega_{t} + \lambda \left ( \Delta \omega_{t} \right )^2 + \rho \left ( u^T \omega_{t}-1 \right )^2$

<br>

**($\omega_{t}$ are the FLOAT variables)** Warning: $\omega_{t}$ is an array of N variables $\omega_{it}$


<br>
<br>

In order to tackle the problem with the Binary Quadratic Model approach, we need to map the arrays of float variables $\omega_{t}$ into binary variables. We do that using a binary fraction expansion:


We map float variables into binary variables using bits.

<br>
<br>

Using $N_{q}$ bits, the coding of $<\omega_{nt}>$ is: 

<br>  


$<\omega_{nt}>=  \sum_{q=0}^{N_{q}-1}2^q x_{ntq}$


<br>
<br>

And if we include the normalization constant K in the codification, we get the following expression for the proportions $\omega_{nt}$:

<br>

$\omega_{nt}= \sum_{q=0}^{N_{q}-1}2^{-q} x_{ntq}$



## Let's start coding
<br>
<br>
<br>

<br>
<br>

# Returns contribution

<br>
<br>

$\sum_{t}\mu _{t}^{T}\omega_{t}$

<br>
<br>

$\mu _{t}$ ------> means\[time, asset\]



<br>

Each scalar variable in the variable vector $\omega_{t}=[\omega_{1t}, ..., \omega_{Nt}]$ is expanded in binary fractions ----->  $\omega_{nt}=\frac{1}{2^0}X_{nt0}+\frac{1}{2^1}X_{nt1}+\frac{1}{2^2}X_{nt2}+\frac{1}{2^3}X_{nt3}  $

<br>

The binary variables are $X_{nt\text{digit}}$ with digit=0,1,2,3 in this particular case.


<br>

Finally,

<br>

$\sum_{t}\mu _{t}^{T}\omega_{t}$ is coded as follows:


<br>

$\sum_{\text{time}}\sum_{\text{asset}}\sum_{\text{digit}}f(time, asset, digit)\text{    }\text{X}_{time, asset, digit}$

<br>
<br>

$f(time, asset, digit) = \frac{-1}{{2}^{\text{digit }}} \text{means}(time, asset)$


<br>
<br>

# Normalization contribution

<br>
<br>

In equation (1), the term $\rho \left ( u^T \omega_{t}-1 \right )^2$ corresponds to a normalization contribution. Let's re-write the expression as follows:

<br>

(2) $\left ( u^T \omega_{t}-1 \right )^2= \left ( \sum_{i}\omega_{ti}-1  \right )^2 = \sum_{i} \omega_{ti}^2 + \sum_{i!=j} \omega_{ti}\omega_{tj}-2 \sum_{i} \omega_{ti}+1 = \sum_{i}\omega_{ti}(\omega_{ti}-2)+ \sum_{i!=j} \omega_{ti}\omega_{tj} +1 $











# Normalization linear contribution

<br>
<br>

$\sum_{t}\sum_{i}\omega_{ti}(\omega_{ti}-2)\text{    }$ --- (binary variables) ---->$\text{    } \sum_{t}\sum_{i}\sum_{d}\frac{1}{2^d}X_{tid}(X_{tid}-2)$



<br>
<br>

They code ...

<br>

`h\[time*n_digits*n_assets + asset*n_digits + digit\] += A * (2**(-digit) - 1) * 2**(-digit)`

<br>

I think the correct code is ... 

<br>

`h\[time*n_digits*n_assets + asset*n_digits + digit\] += A * (2**(-digit) - 2) * 2**(-digit)`

# Normalization quadratic contribution

<br>
<br>

$\sum_{i!=j} \omega_{ti}\omega_{tj} \text{    }$ --- (binary variables) ---->$\text{    } \sum_{t}\sum_{i}\sum_{d1}\sum_{d2}\frac{1}{2^{d1}}\frac{1}{2^{d2}}X_{tid1}X_{tid2}$



<br>
<br>

They code ...

<br>

`J[(coord1,coord2)] = A * 2**(-digit1) * 2**(-digit2)`



# Covariance contribution

<br>
<br>

The term of equation (1) corresponding to the variance is:

<br>

$0.5\gamma \omega_{t}^{T}\sum_{t}\omega_{t}\text{     }$ 


<br>

**warning:** here $\Sigma_{t}$ is a matrix not a summation symbol


<br>
<br>


They use `covs[time, asset1, asset2]=np.random.rand(n_intervals, n_assets, n_assets)`  for the matrix $\sum_{t}$ 

So, we've got:

<br>

$\omega_{t}^{T}\sum_{t}\omega_{t}  \text{    }$ --- (binary variables) ---->$\text{    }\sum_{t,i,j,d1,d2}\text{covs[t,i,j]}\frac{1}{2^{d1}}\frac{1}{2^{d2}}X_{t,i,d1}X_{t,j,d2}$



<br>

A covariance matrix is a **symmetric** one. By construction, `covs[time, asset1, asset2]=np.random.rand(n_intervals, n_assets, n_assets)`  is not symmetric. We can cast the scenario as a symmetric case using only the upper triangular block of the matrix.


<br>


<br>

$\omega_{t}^{T}\sum_{t}\omega_{t}  \text{    }$ --- (binary variables) ---->$\text{    }\sum_{t,j<i,d1,d2}\text{covs[t,i,j]}\frac{1}{2^{d1}}\frac{1}{2^{d2}}X_{t,i,d1}X_{t,j,d2}$



<br>

#### Error

They code ... 


`if asset1 >= asset2 and digit1 >= digit2: continue`

<br>

I think it should be ...

`if asset1 >= asset2: continue`

